## Trabalho Prático 2: Realidade Aumentada

 
Nome: Arthur Pontes Nader\
Matrícula: 2019022294


O Trabalho Prático 2 da disciplina Introdução a Computação Visual consistiu na detecção e localização de alvos nos frames de um vídeo e na inserção de objetos tridimensionais sobre esses alvos. Os seguintes links correspondem a vídeos no YouTube com a explicação do trabalho e mostrando os resultados obtidos:

Explicação do trabalho: https://www.youtube.com/watch?v=ok7CfWbfWew


Detecção dos alvos: https://www.youtube.com/watch?v=CLn6HYKVIDo


Realidade aumentada: https://www.youtube.com/watch?v=RH9U3x8m8P0



Para realização do trabalho, utilizou-se as seguintes bibliotecas:

In [1]:
import numpy as np
import cv2
from PIL import Image
import pygame

from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

from objloader import *

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


## 1) Calibração da câmera

Para realizar a calibração da câmera, utilizou-se a ferramenta Octave 5.2.0 e o toolbox de Jean-Yves-Bouguet disponibilizada na seguinte página: https://github.com/nghiaho12/camera_calibration_toolbox_octave. 

Usou-se 7 frames do vídeo que forneciam uma medida da cena a partir de diferentes posições e orientações da câmera. Os resultados podem ser visualizados a seguir, sendo que ao final do notebook há um apêndice com os resultados oficiais fornecidos pelo Octave.

In [2]:
centro_otico_x = 306.31489
centro_otico_y = 235.42074 
distancia_focal_x = 413.19647
distancia_focal_y = 412.39518
skew = 0.00
coeficientes_distorcao = np.float64([[0.08027, -0.19506, -0.00007, -0.00781, 0.00000]])
pixel_erro = [0.16675, 0.20123]

Com esses valores, pode-se gerar a seguinte matriz de parâmetros intrínsecos:

In [3]:
matriz_intrinsecos = np.float64([[distancia_focal_x, skew, centro_otico_x],
                                [0, distancia_focal_y, centro_otico_y],
                                [0,0,1]])

## 2) Detecção dos alvos

A função gerarAlvos recebe como parâmetro o alvo e, após convertê-lo para uma imagem binária, o rotaciona diversas vezes para que posteriormente seja possível comparar se uma imagem corresponde ao alvo em uma orientação diferente.

In [4]:
def gerarAlvos(img):
    
    alvo_cinza = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    alvo_binario = cv2.threshold(alvo_cinza, 108, 255, cv2.THRESH_BINARY)[1]
    
    alvo1 = alvo_binario
    alvo2 = np.rot90(alvo1)
    alvo3 = np.rot90(alvo2)
    alvo4 = np.rot90(alvo3)
    
    return alvo1, alvo4, alvo3, alvo2

A função detectarQuadrilateros recebe um frame do video e, após convertê-lo para uma imagem binária, detecta os contornos presentes. A seguir, uma série de filtragens, como relacionada a área e o número de vértices do contorno, são realizadas para que somente se tenha os resultados mais relevantes. Como se está interessado em idenficar um quadrilátero, só serão retornados os contornos que possuírem 4 vértices.

In [5]:
def detectarQuadrilateros(frame):

    img_cinza = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, image_bin = cv2.threshold(img_cinza, 127, 255, 0)

    deteccoes, hierarquia = cv2.findContours(image_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    deteccoes = [det for det in deteccoes if cv2.contourArea(det) < 20000.0 and cv2.contourArea(det) > 1500.0]
    deteccoes = sorted(deteccoes, key = cv2.contourArea, reverse = True)[:5]
    
    quadrilateros = []
    for det in deteccoes:
        quad = cv2.approxPolyDP(det, 0.02 * cv2.arcLength(det, True), True)
        if len(quad) == 4:
            quadrilateros.append(quad)
    
    return quadrilateros
    

Já a função detectarAlvos recebe os quadriláteros detectados no frame e, por meio da realização de uma homografia, retifica e gera uma nova imagem com a porção do frame correspondente ao quadrilátero. Assim, pode-se usar um algoritmo de Template Matching para confirmação se o quadrilátero realmente é um alvo, e caso seja, qual é a sua orientação. 
Os seguintes métodos podem ser passados como parâmetro na chamada da função para realizar o Template Matching: 
* TM_CCORR_NORMED
* TM_CCOEFF_NORMED
* TM_SQDIFF_NORMED

Por padrão, utilizou-se o método TM_CCORR_NORMED, pois esse método apresentou resultados satisfatórios na detecção do alvo.

Para diferenciar cada um dos alvos na imagem, utilizou-se uma manipulação direta dos pixels, já que se pecebeu que o alvo 1 só aparecia na metade esquerda e o alvo 2 só ocorria na parte superior do video. Assim, o alvo 1 será contornado de verde, o 2 de azul e o 3 de amarelo. Essa diferenciação será usada posteriormente para determinar o sentido de rotação do Pikachu.

In [6]:
def detectarAlvo(x, y, quadrilateros, frame, alvos, metodo = cv2.TM_CCORR_NORMED, limite = 0.75):
    
    alvos_localizados = []
    
    for quad in quadrilateros:
        
        coordenadas = [[0,0],[x, 0],[x,y],[0, y]]
        matriz_homografia, _ = cv2.findHomography(np.float64(quad), np.float64(coordenadas))
        alvo_retificado = cv2.warpPerspective(frame, matriz_homografia, (x,y))
        
        img_cinza = cv2.cvtColor(alvo_retificado, cv2.COLOR_BGR2GRAY)
        ret, image_bin = cv2.threshold(img_cinza, 127, 255, 0)
        
        coords_x = [quad[k][0][0] < frame.shape[1]//2 for k in range(4)]
        coords_y = [quad[k][0][1] < frame.shape[0]//2 for k in range(4)]
        
        alvo_1 = coords_x[0] and coords_x[1] and coords_x[2] and coords_x[3]
        alvo_2 = coords_y[0] and coords_y[1] and coords_y[2] and coords_y[3]
            
        
        for j in range(len(alvos)):
            resultado = cv2.matchTemplate(image_bin, alvos[j], metodo)
            
            if metodo == cv2.TM_SQDIFF_NORMED:
                resultado = 1.0 - resultado
            
            if resultado > limite:
                if alvo_1:
                    cv2.drawContours(frame, [quad], -1, (55,200, 20), 2)
                    alvos_localizados.append((quad,"alvo1",j))
                elif alvo_2:
                    cv2.drawContours(frame, [quad], -1, (200,55, 20), 2)
                    alvos_localizados.append((quad,"alvo2",j))
                else:
                    cv2.drawContours(frame, [quad], -1, (55,200, 200), 2)
                    alvos_localizados.append((quad,"alvo3",j))
                
                proximo_vertice = (j+1)%4
                x_centro = int((quad[0][0][0]+quad[2][0][0])/2)
                y_centro = int((quad[0][0][1]+quad[2][0][1])/2)
                x_frente = int((quad[j][0][0]+quad[proximo_vertice][0][0])/2)
                y_frente = int((quad[j][0][1]+quad[proximo_vertice][0][1])/2)
                cv2.arrowedLine(frame, (x_centro, y_centro), (x_frente, y_frente), (20,55,200), 2)
                
                break
            
    return frame, alvos_localizados

## 3) Obtenção da pose da câmera

A função encontrarPose recebe a matriz de parâmetros intrinsecos, os coeficientes de distorção, a localização do alvo na cena e a coordenadas de seus vértices no mundo. Assim por meio da utilização do método solvePnP, gera-se um vetor de rotação e um vetor de translação, que, após algumas manipulações, são transformados na matriz de parâmetros extrínsecos.

In [7]:
def encontrarPose(intrinsecos, mundo, distorcao, alvo):

    alvo_aux = np.float64([ele[0] for ele in alvo])
    
    ret, vetor_rotacao, vetor_translacao = cv2.solvePnP(mundo, alvo_aux, intrinsecos, distorcao)
    
    matriz_extrinsecos, _ = cv2.Rodrigues(vetor_rotacao)
    matriz_extrinsecos = np.concatenate([matriz_extrinsecos, vetor_translacao], axis=1)
    matriz_extrinsecos = np.concatenate((matriz_extrinsecos, [[0.0,0.0,0.0,1.0]]), axis=0)
    
    return matriz_extrinsecos

Ao final desse notebook, há um código que mostra os resultados de tudo que foi feito até essa parte do trabalho. Nesse código, a função encontrarPose é um pouco diferente para que se possa observar a projeção de um ponto na cena utilizando a matriz gerada pela função encontrarPose. (vale a pena conferir, é bem legal)

## 4) Renderização do cubo e do Pikachu

A função object3D a seguir recebe como parâmetros o objeto a ser renderizado, a matriz gerada em encontrarPose e o número do alvo em questão. 
Primeiramente, a matriz é transformada para se adequar ao sistema de coordenadas da OpenGL. Então, renderiza-se um cubo no local e uma seta que indica a frente do alvo. \
Após isso, o Pikachu é renderizado no mesmo local. Só que, se o alvo identificado for o número 1 ou o 3, o Pikachu será rotacionado no sentido anti-horário. Caso seja o 2, ele é rotacionado no sentido horário

In [8]:
def object3D(obj, matriz, num_alvo):
    
    matriz[1,:] = -matriz[1,:]
    matriz[2,:] = -matriz[2,:]
    matriz = matriz.T
    matriz = matriz.flatten() 
    
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glLoadMatrixf(matriz)
    glLineWidth(2.15)
    glBegin(GL_LINES)
    glVertex3f(2.5,2.5,0)
    glVertex3f(2.5,10.5,0)
    glVertex3f(2.5,10.5,0)
    glVertex3f(1.7,8.7,0)
    glVertex3f(2.5,10.5,0)
    glVertex3f(3.3,8.7,0)
    glEnd()
    glutWireCube(7.8)
    
    global rotacao
    glLoadIdentity()
    glLoadMatrixf(matriz)
    
    if num_alvo == "alvo1" or num_alvo == "alvo3":
        glRotate(rotacao, 0, 0, 1)
    elif num_alvo == "alvo2":
        glRotate(-rotacao, 0, 0, 1)
        
    glScalef(2.31, 2.31, 2.31)
    glCallList(obj.gl_list)

## 5) Funções da OpenGL

As funções a seguir são usadas para inicializar, definir, habilitar e limpar alguns paramêtros de execução da OpenGL.

In [9]:
def initOpenGL(dimensions):

    (width, height) = dimensions
    
    glClearColor(0.0, 0.0, 0.0, 0.0)
    glClearDepth(1.0)

    glEnable(GL_DEPTH_TEST)

    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
 
    fovy = 2*np.arctan(0.5*height/distancia_focal_y)*180/np.pi;
    aspect = (width*distancia_focal_x)/(height*distancia_focal_y);
    gluPerspective(fovy, aspect, 0.1, 100.0)
    
def idleCallback():
    
    glutPostRedisplay()

Já a função aplicarFundo recebe um frame do video e, após criar uma identificação para a textura, realiza as operações que irão mapear o frame como fundo da cena em que os objetos serão renderizados. Em seguida a textura criada é deletada. (reparou-se que, se isso não for feito, parece que há um vazamento de memória, fazendo com que a exibição dos resultados na tela fique cada vez mais lenta).

In [10]:
def aplicarFundo(fundo, largura, altura):
    
    fundo = cv2.flip(fundo, 0)
    fundo = cv2.cvtColor(fundo, cv2.COLOR_BGR2RGB)
    
    nome_textura = glGenTextures(1)
    
    glBindTexture(GL_TEXTURE_2D, nome_textura)
    glTexImage2D(GL_TEXTURE_2D,0,GL_RGB,largura,altura,0,GL_RGB, GL_UNSIGNED_BYTE, fundo)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER,GL_NEAREST)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER,GL_NEAREST)
    
    glMatrixMode(GL_PROJECTION)
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(-1, 1, -1, 1)
    
    glBegin(GL_QUADS)
    glTexCoord2f(0.0, 0.0); glVertex3f(-1.0, -1.0, 0.0)
    glTexCoord2f(1.0, 0.0); glVertex3f(1.0, -1.0, 0.0)
    glTexCoord2f(1.0, 1.0); glVertex3f(1.0, 1.0, 0.0)
    glTexCoord2f(0.0, 1.0); glVertex3f(-1.0, 1.0, 0.0) 
    glEnd()
    glPopMatrix()
    
    glDeleteTextures(1, nome_textura)

A função displayCallback é acionada continuamente por glutMainLoop. Então, será nela que deverão ser chamados os procedimentos de carregamento do modelo 3D do Pikachu, leitura de frame do vídeo, aplicação do frame como fundo da cena, detecção dos alvos, cálculo da pose e renderização dos objetos.

In [11]:
def displayCallback():
    
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    # carregar o modelo 3D do Pikachu
    obj = OBJ("Pikachu.obj", swapyz=True)
    
    # habilita o uso de texturas (o Pikachu tem textura)
    glEnable(GL_TEXTURE_2D)
    
    global captura
    ret, frame = captura.read()
    
    if ret:    
        aplicarFundo(frame, frame.shape[1], frame.shape[0])
        quads = detectarQuadrilateros(frame)
        _, alvos_localizados = detectarAlvo(alvo.shape[0], alvo.shape[1], quads, frame, alvos)
    
        for alvo_loc in alvos_localizados:
            mundo_aux = coordenadas_mundo.copy()
            for i in range(alvo_loc[2]):
                mundo_aux = np.concatenate(([mundo_aux[3].copy()],mundo_aux), axis=0)
                mundo_aux = np.delete(mundo_aux, 4, 0)

            pose = encontrarPose(matriz_intrinsecos, mundo_aux, coeficientes_distorcao, alvo_loc[0])
            object3D(obj, pose, alvo_loc[1])
        
        global rotacao
        rotacao += 5
        rotacao = rotacao%360
            
    glutSwapBuffers()

## 6) Execução do programa

Agora, com todas as funções definidas, as variáveis a seguir são declaradas para serem utilizadas ao longo da execução. Elas se referem ao carregamento e geração dos alvos, ao sistemas de coordenadas no mundo correspondente a um alvo detectado, à rotação do Pikachu e ao carregamento do vídeo de entrada. Em seguida, inicia-se a execução do programa.

In [12]:
alvo = cv2.imread("alvo.jpg")
alvos = gerarAlvos(alvo)
coordenadas_mundo = np.float64([[0,5,0],[5,5,0],[5,0,0],[0,0,0]])
rotacao = 0
captura = cv2.VideoCapture('entrada.mp4')

if __name__ == '__main__':
    dimensions = (640, 480)
    glutInit()
    glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
    glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
    glutInitWindowSize(*dimensions)
    window = glutCreateWindow(b'TP2 - Realidade Aumentada')
    
    initOpenGL(dimensions)
    
    glutDisplayFunc(displayCallback)
    glutIdleFunc(idleCallback)
    
    glutMainLoop()

## 7) Conclusão

A realização desse trabalho prático foi uma boa oportunidade de colocar em prática os conceitos aprendidos durante a segunda parte da disciplina Introdução a Computação Visual. A sua implementação possibitou fixar vários dos métodos de computação visual vistos durante as aulas, como homografia, template matching, estimação de pose, renderização de objetos e mapeamento de textura.

## 8) Referências Bibliográficas

### Livros
* SOLEM, J, E. Programming Computer Vision with Python, O'Reilly, 2012. 300 p.
* WRIGHT, R, S. Jr.; SWEET, M. OpenGL SuperBible. 2nd ed. Indianapolis, Indiana: Waite Group Press, 2000. 696 p.

### Sites

##### Calibração da câmera
* https://learnopencv.com/camera-calibration-using-opencv/
* https://github.com/nghiaho12/camera_calibration_toolbox_octave

##### Homografia
* https://learnopencv.com/homography-examples-using-opencv-python-c/

##### Detecção do alvo
* https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html
* https://docs.opencv.org/4.x/d4/dc6/tutorial_py_template_matching.html
* https://docs.opencv.org/3.4/d9/d0c/group__calib3d.html#ga61585db663d9da06b68e70cfbf6a1eac

##### Numpy
* https://numpy.org/doc/stable/reference/constants.html
* https://numpy.org/doc/stable/reference/generated/numpy.arctan.html

##### OpenGL
* https://www.inf.pucrs.br/~manssour/OpenGL/Introducao.html
* http://pyopengl.sourceforge.net/documentation/manual-3.0/glDeleteTextures.html
* https://www.inf.pucrs.br/~pinho/CG/Aulas/OpenGL/Texturas/MapTextures.html
* https://itecnote.com/tecnote/opencv-reference-coordinate-system-changes-between-opencv-opengl-and-android-sensor/

## 9) Apêndices

### Resultados da calibração

Calibration results after optimization (with uncertainties):

Focal Length:          fc = [ 413.19647   412.39518 ] +/- [ 10.93237   9.94818 ]\
Principal point:       cc = [ 306.31489   235.42074 ] +/- [ 4.84831   8.18127 ]\
Skew:             alpha_c = [ 0.00000 ] +/- [ 0.00000  ]   => angle of pixel axes = 90.00000 +/- 0.0000
0 degrees\
Distortion:            kc = [ 0.08027   -0.19506   -0.00007   -0.00781  0.00000 ] +/- [ 0.05616   0.389
20   0.00584   0.00498  0.00000 ]\
Pixel error:          err = [ 0.16675   0.20123 ]

Note: The numerical errors are approximately three times the standard deviations (for reference).

### Resultados da identificação do alvo na cena

In [13]:
alvo = cv2.imread("alvo.jpg")
alvos = gerarAlvos(alvo)

coordenadas_mundo = np.float64([[0,5,0],[5,5,0],[5,0,0],[0,0,0]])

In [14]:
def encontrarPose2(intrinsecos, mundo, distorcao, frame, alvo):

    alvo_aux = np.float64([ele[0] for ele in alvo])
    
    ret, vetor_rotacao, vetor_translacao = cv2.solvePnP(mundo, alvo_aux, intrinsecos, distorcao)
    
    matriz_extrinsecos, _ = cv2.Rodrigues(vetor_rotacao)
    destino, _ = cv2.projectPoints(np.array([(2.5, 2.5, 5.0)]), vetor_rotacao, vetor_translacao,intrinsecos, distorcao)
    
    for ponto in alvo_aux:
        cv2.circle(frame, (int(ponto[0]), int(ponto[1])), 3, (0,0,255), -1)
    
    p1 = int((alvo[0][0][0]+alvo[2][0][0])/2), int((alvo[0][0][1]+alvo[2][0][1])/2)
    p2 = int(destino[0][0][0]), int(destino[0][0][1])
    cv2.line(frame, p1, p2, (255,100,60), 2)
    
    matriz_extrinsecos = np.concatenate([matriz_extrinsecos, vetor_translacao], axis=1)
    matriz_extrinsecos = np.concatenate((matriz_extrinsecos, [[0.0,0.0,0.0,1.0]]), axis=0)
    
    return frame, matriz_extrinsecos

In [15]:
captura = cv2.VideoCapture('entrada.mp4')

while(captura.isOpened()):
    ret, frame = captura.read()
    if ret == True:

        quads = detectarQuadrilateros(frame)
        deteccao, alvos_localizados = detectarAlvo(alvo.shape[0], alvo.shape[1], quads, frame, alvos)

        for alvo_loc in alvos_localizados:
            mundo_aux = coordenadas_mundo.copy()
            for i in range(alvo_loc[2]):
                mundo_aux = np.concatenate(([mundo_aux[3].copy()],mundo_aux), axis=0)
                mundo_aux = np.delete(mundo_aux, 4, 0)

            deteccao, _ = encontrarPose2(matriz_intrinsecos, mundo_aux, coeficientes_distorcao, deteccao, alvo_loc[0])
            
        cv2.imshow("Deteccao dos alvos", deteccao)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    else:
        break
        
captura.release()
cv2.destroyAllWindows()